제작자: Chanjun Park (박찬준)<br>
소속: Korea University NLP&AI Lab (고려대학교 자연어처리&인공지능 연구실)<br>

튜토리얼 명: Keras로 배우는 Artificial Neural Network

<1>간단한 인공신경망 실습을 진행해봅시다.
숫자 5개 중 2개를 학습해 나머지 3개를 예측해봅시다.

In [0]:
import keras
import numpy

x=numpy.array([0,1,2,3,4]) #숫자 5개 선언
y=x*2+1 #y값 정의

model=keras.models.Sequential() #인스턴스 만들기
model.add(keras.layers.Dense(1,input_shape=(1,)))#계층 추가/입력은 1개

model.compile('SGD','mse') #어떻게 학습할지 적기 (최적화알고리즘, 손실함수)

model.fit(x[:2],y[:2],epochs=1000,verbose=0) #실제 학습하기 , verbose가 0일경우 학습진행 상황 표시 X, 2개는 학습 진행

print('Targets: ',y[2:]) #3개는 예측을 진행
print('Predictions:',model.predict(x[2:]).flatten()) #실제 예측하기
print('Predictions:',model.predict(x[2:])) #flatten을 빼면 각각 표시됨.

ANN (인공신경망)
얕은 신경망 shallow Neural Network

핵심은 입력,은닉,출력 계층으로 이루어져 있다 !

1단계: 입력이 들어오면 가중치 행렬을 곱해준다. <br>
2단계: 이를 은닉 행렬의 입력으로 넣어준다.<br>
3단계: 은닉 계층에서는 입력된 값에 활성화 함수를 적용한다. 이는 비선형성이라는 특징을 부여한다. <br>
4단계: 여기에 또 다시 가중치 행렬을 곱하고 출력계층으로 보내준다.<br>
5단계: 출력계층은 마찬가지로 활성화 함수를 적용하고 최종적으로 출력값을 내보낸다.<br>
6단계: 만약 분류 문제일 경우 소프트맥스 연산을 사용한다. 

분류와 회귀<br>
분류: 0~9 숫자 중 무엇이냐, 스팸 햄 메일 중 어떤거?<br>
회귀: 일주일간 온도가 이랬는데 내일 온도는? 연속성

Mnist 예제로 학습해보기

In [0]:
import numpy as np
from keras import datasets
from keras.utils import np_utils
from keras import layers,models
import matplotlib.pyplot as plt

from collections import defaultdict


#이곳은 하이퍼파라미터를 설정하는 곳 입니다.
HPARAMS = defaultdict(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy',
    n_epoch=10,
    batch_size=100,
    validation_split=0.2 
)

hparams = type('', (object,), HPARAMS)() # dict to class

class ANN_MNIST(models.Sequential):
  def __init__(self,Nin,Nh,Nout):
    super().__init__()
    self.add(layers.Dense(Nh,activation='relu',input_shape=(Nin,)))
    self.add(layers.Dense(Nout,activation='softmax'))
    self.compile(loss=hparams.loss,optimizer=hparams.optimizer,metrics=[hparams.metrics])
    
def one_hot(X_test,Y_test): 
  X_test=np_utils.to_categorical(X_test)
  Y_test=np_utils.to_categorical(Y_test)
  
  return X_test,Y_test

def make_2d(X_train,X_test):
  L,H,W=X_train.shape
  X_train=X_train.reshape(-1, H*W)
  X_test=X_test.reshape(-1, H*W)
  
  X_train=X_train/255.0
  X_test=X_test/255.0
  
  
  return X_train,X_test

def data_load():
  (X_train,Y_train),(X_test,Y_test)=datasets.mnist.load_data()
  
  Y_train,Y_test=one_hot(Y_train,Y_test)
  
  X_train,X_test=make_2d(X_train,X_test)
  
  return (X_train, X_test, Y_train, Y_test)

def plot_loss(history):
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train','Test'],loc=0) #두선의 이름 표시

def plot_acc(history):
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train','Test'],loc=0)
  
def main():
  Nin=784
  Nh=100
  Nout=10
  
  (X_train,X_test,Y_train,Y_test)=data_load()
  
  
  model=ANN_MNIST(Nin,Nh,Nout)
  
  history= model.fit(X_train,Y_train,epochs=hparams.n_epoch,batch_size=hparams.batch_size,validation_split=hparams.validation_split)
  performance_test=model.evaluate(X_test,Y_test,batch_size=hparams.batch_size)
  print('Test Loss and Accuracy -> ',performance_test)
  
  plot_loss(history)
  plt.show()
  

if __name__ == '__main__':
  main()


회귀 문제를 이제 풀어보자
보스턴 집값 문제



In [0]:
from keras import layers,models
from keras import datasets
from sklearn import preprocessing

class ANN(models.Model):
  def __init__(self,Nin,Nh,Nout):
    hidden=layers.Dense(Nh)
    output=layers.Dense(Nout)
    relu=layers.Activation('relu')
    
    
    #연결 정의
    x=layers.Input(shape=(Nin,))
    h=relu(hidden(x))
    y=output(h)
    
    super().__init__(x,y)
    
    self.compile(loss='mse',optimizer='sgd')

def data_load():
  (X_train,y_train),(X_test,y_test)=datasets.boston_housing.load_data()
  scaler=preprocessing.MinMaxScaler()
  X_train=scaler.fit_transform(X_train)
  X_test=scaler.fit_transform(X_test)
  
  return X_train,y_train,X_test,y_test

def main():
  Nin=13
  Nh=5
  Nout=1
  
  X_train,y_train,X_test,y_test=data_load()
  
  model=ANN(Nin,Nh,Nout)
  history=model.fit(X_train,y_train,epochs=100,batch_size=100,validation_split=0.2,verbose=2)
  
  
  performance_test=model.evaluate(X_test,y_test,batch_size=100)
  print('Test Loss -> {:.2f}'.format(performance_test))


if __name__ == '__main__':
  main()
  

ANN을 지나 이제 DNN을 배워보자
ANN은 은닉의 출력이 출력의 입력으로
DNN은 은닉의 출력이 새로운 은닉의 입력으로
단지 그 차이일 뿐.......

In [0]:
from keras import layers,models
from keras import datasets
from keras.utils import np_utils

from collections import defaultdict
import numpy as np



#이곳은 하이퍼파라미터를 설정하는 곳 입니다.
HPARAMS = defaultdict(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy',
    n_epoch=10,
    batch_size=100,
    validation_split=0.2 
)

hparams = type('', (object,), HPARAMS)() # dict to class


class DNN(models.Sequential):
  def __init__(self,Nin,Nh_1,Nout):
    super().__init__()
    
    #입력 계층의 정의는 첫번째 hidden을 정의할 때 같이 이루어지게 된다.
    self.add(layers.Dense(Nh_1[0],activation='relu',input_shape=(Nin,),name='Hidden-1'))
    self.add(layers.Dropout(0.2))
    
    #제2계층부터는 케라스가 자동으로 현재 계층의 입력 노드 수를 앞에 나온 은닉계층의 출력 수로 설정해준다.
    self.add(layers.Dense(Nh_1[1],activation='relu',name='Hidden-2'))
    self.add(layers.Dropout(0.2))
    self.add(layers.Dense(Nout,activation='softmax'))
    
    self.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


def one_hot(X_test,Y_test): 
  X_test=np_utils.to_categorical(X_test)
  Y_test=np_utils.to_categorical(Y_test)
  
  return X_test,Y_test

def make_2d(X_train,X_test):
  L,H,W=X_train.shape
  X_train=X_train.reshape(-1, H*W)
  X_test=X_test.reshape(-1, H*W)
  
  X_train=X_train/255.0
  X_test=X_test/255.0
  
  
  return X_train,X_test

def data_load():
  (X_train,Y_train),(X_test,Y_test)=datasets.mnist.load_data()
  
  Y_train,Y_test=one_hot(Y_train,Y_test)
  
  X_train,X_test=make_2d(X_train,X_test)
  
  return (X_train, X_test, Y_train, Y_test)   
    
def main():
  #기본 파라미터 설정
  Nin=784
  Nh_1=[100,50] #2개의 은닉층
  Nout=10
  
  (X_train,X_test,Y_train,Y_test)=data_load()
  
  model=DNN(Nin,Nh_1,Nout)
  
  history= model.fit(X_train,Y_train,epochs=hparams.n_epoch,batch_size=hparams.batch_size,validation_split=hparams.validation_split)
  performance_test=model.evaluate(X_test,Y_test,batch_size=hparams.batch_size)
  print('Test Loss and Accuracy -> ',performance_test)
  
  plot_loss(history)
  plt.show()
  

if __name__ == '__main__':
  main()


이번에는 컬러 이미지를 분류해보자.
RGB이니깐 32 * 32 * 3 이다.


In [0]:
from keras import layers,models
from keras import datasets
from keras.utils import np_utils
import numpy as np
  
from collections import defaultdict
import numpy as np

#이곳은 하이퍼파라미터를 설정하는 곳 입니다.
HPARAMS = defaultdict(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy',
    n_epoch=10,
    batch_size=100,
    validation_split=0.2 
)

hparams = type('', (object,), HPARAMS)() # dict to class


class DNN(models.Sequential):
  def __init__(self,Nin,Nh_1,pd_l,Nout):
    super().__init__()
    
    #입력 계층의 정의는 첫번째 hidden을 정의할 때 같이 이루어지게 된다.
    self.add(layers.Dense(Nh_1[0],activation='relu',input_shape=(Nin,),name='Hidden-1'))
    self.add(layers.Dropout(pd_l[0]))
    
    #제2계층부터는 케라스가 자동으로 현재 계층의 입력 노드 수를 앞에 나온 은닉계층의 출력 수로 설정해준다.
    self.add(layers.Dense(Nh_1[1],activation='relu',name='Hidden-2'))
    self.add(layers.Dropout(pd_l[1]))

    self.add(layers.Dense(Nout,activation='softmax'))
    
    self.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


def data_load():
  (X_train,y_train),(X_test,y_test)=datasets.cifar10.load_data()

  Y_train=np_utils.to_categorical(y_train)
  y_test=np_utils.to_categorical(y_test)
  
  L,W,H,C=X_train.shape #50000 32 32 3
  #print(L,H,H,C)
  X_train=X_train.reshape(-1,W*H*C)
  X_test=X_test.reshape(-1,W*H*C)

  return (X_train,X_test,Y_train,Y_test)

def main():
  #기본 파라미터 설정
  Nh_1=[100,50] #2개의 은닉층
  pd_l=[0.0,0.0]
  Nout=10
  
  (X_train,X_test,Y_train,Y_test)=data_load()
  
  model=DNN(X_train.shape[1],Nh_1,pd_l,Nout) #입력, 은닉개수,드롭아웃 비율, 출력
  
  history= model.fit(X_train,Y_train,epochs=hparams.n_epoch,batch_size=hparams.batch_size,validation_split=hparams.validation_split)
  performance_test=model.evaluate(X_test,Y_test,batch_size=hparams.batch_size)
  print('Test Loss and Accuracy -> ',performance_test)
 
  

if __name__ == '__main__':
  main()


이제는 CNN입니다.

In [0]:
from keras import layers,models
from keras import datasets
from keras.utils import np_utils
from keras import backend
import keras
from collections import defaultdict
import numpy as np




class CNN(models.Sequential):
  def __init__(self,input_shape,num_classes):
    super().__init__()
    
    self.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape)) #커널개수, 커널사이즈, 활성화함수, 입력 
    self.add(layers.Conv2D(64,(3,3),activation='relu'))
    self.add(layers.MaxPooling2D(pool_size=(2,2))) #2x2 셀로 묶어서 가장 큰값을 내보낸다.
    self.add(layers.Dropout(0.25))
    
    self.add(layers.Flatten())#완전연결 계층으로 들어간다.
   
    self.add(layers.Dense(128,activation='relu')) #은닉 계층은 길이가 128로
    self.add(layers.Dropout(0.5))
    self.add(layers.Dense(num_classes,activation='softmax'))
    
    self.compile(loss=keras.losses.categorical_crossentropy,optimizer='rmsprop',metrics=['accuracy'])
    

class DATA():
  def __init__(self):
    num_classes=10
    (x_train,y_train),(x_test,y_test)=datasets.mnist.load_data()
    print(x_train.shape) #60000,784 (원래)
    
    #img_rows,img_cols=X_train.shape[1:]
    img_rows,img_cols=int(28),int(28) 
    print(img_rows,img_cols)
    
    if backend.image_data_format()=='channels_first':
      x_train=x_train.reshape(x_train.shape[0],1,img_rows,img_cols)
      x_test=x_test.reshape(x_test.shape[0],1,img_rows,img_cols)
      input_shape=(1,img_rows,img_cols) #컬러 채널을 임의로 첫번째로 해준다.
    else:
      x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
      x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
      input_shape=(img_rows,img_cols,1) #마지막에 컬러 채널을 임의로 첫번째로 해준다.
     
    
    x_train=x_train.astype('float32')
    x_test=x_test.astype('float32')
    x_train/=255
    x_test/=255
    
    y_train=keras.utils.to_categorical(y_train,num_classes)
    y_test=keras.utils.to_categorical(y_test,num_classes)
    
    
    self.input_shape=input_shape
    self.num_classes=num_classes
    
    self.x_train,self.y_train=x_train,y_train
    self.x_test,self.y_test=x_test,y_test
    
       

def main():
  batch_size=128
  epochs=10
  
  data=DATA()
  model=CNN(data.input_shape,data.num_classes)
  
  history= model.fit(data.x_train,data.y_train,batch_size=batch_size,epochs=epochs,validation_split=0.2)
  
  score=model.evaluate(data.x_test,data.y_test)
  print()
  print('Test Loss',score[0])
  print('Test Accuracy',score[1])
  

if __name__ == '__main__':
  main()
